# December 24, 2024

https://adventofcode.com/2024/day/24

In [87]:
import re

In [37]:
DEBUG = False
def dprint( *args ):
    if DEBUG:
        return print(*args)


In [38]:
test_str1 = f'''x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02'''
test_str1 = test_str1.split("\n")

test_str2 = f'''x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj'''
test_str2 = test_str2.split("\n")

In [48]:
fn = "../data/2024/24.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz_str = [x.strip() for x in text]

# Part 1

In [40]:
class Gate:
    def __init__(self, logic, parents ):
        self.logic = logic
        if logic == "const":
            self.parents = None
            self.value = parents
        else:
            self.parents = parents
            self.value = None

    def set_value(self, val0, val1):
        if self.logic == "OR":
            self.value = max(val0, val1)
        elif self.logic == "XOR":
            self.value = int( val0 != val1 )
        elif self.logic == "AND":
            self.value = min(val0, val1)


In [41]:
def parse_input( text ):
    wires = dict()

    for line in text:
        if ":" in line:
            nombre = line[:3]
            val = int(line[-1])
            wires[nombre] = Gate("const", val)

        if len(line) == 0:
            continue

        if "-" in line:
            pieces = line.split(" ")
            par1 = pieces[0]
            par2 = pieces[2]
            logic = pieces[1]
            nombre = pieces[4]
            wires[ nombre ] = Gate( logic, [par1, par2] )
    return wires


In [43]:
def solve_gates( puzz ):
    done = False
    while not done:
        done = True
        for name, gate in puzz.items():
            if gate.value is None:
                parents = gate.parents
                val0 = puzz[parents[0]].value
                val1 = puzz[parents[1]].value
                if val0 is not None and val1 is not None:
                    dprint("setting value for", name)
                    gate.set_value( val0, val1 )
                else:
                    done = False

In [44]:
def part1( text ):
    puzz = parse_input(text)
    solve_gates(puzz)
    
    tot = 0
    for name, gate in puzz.items():
        if name[0] == "z" and gate.value == 1:
            tot += (2 ** int(name[1:3]))

    return tot


In [45]:
part1( test_str1 )

4

In [46]:
part1( test_str2 )

2024

In [49]:
part1( puzz_str )

47666458872582

# Part2

In [85]:
def read_gates( text ):
    gates = dict()
    for line in text:
        if ":" in line or len(line) == 0:
            continue

        if "-" in line:
            pieces = line.split(" ")
            par1 = pieces[0]
            par2 = pieces[2]
            logic = pieces[1]
            nombre = pieces[4]
            gates[ nombre ] = (logic, par1, par2) 

    return gates

Process to calculate a value for z[i], i >= 2

1. x[i-1] AND y[i-1] = carry[i] (Check if the prior digits are both 1 causing a carry)
2. x[i] XOR y[i] = digit[i] (Check if the current digits would result in 0 or 1 before accounting for carries)
3. both[i-1] AND digit[i-1] = chain[i] (Check if the prior digits caused a double carry from the digits twice prior)
4. chain[i] OR carry[i] = both[i] (Check if the regular carry or chain carry is adding a 1 here)
5. digit[i] XOR both[i] = z[i] (Final value for the digit if exactly one of these is adding a 1, if both are 1s it will carry to the next place)

In [198]:
def checkxy( spec ):
    '''check if the input wires are x[nn] and y[nn]'''
    # X inputs can only appear alongside the matching y input, so we only need to check one.
    return re.fullmatch("(x|y)(\d\d)", spec[1])

def identify_gates( gates ):
    # identifies all the parts of the logic, assuming it's correct.
    # we'll verify later to id the broken bits
    lenz = 1 + max( [int(k[1:]) for k in gates if k[0] == "z"] )

    carry = [""] * lenz
    digit = [""] * lenz
    chain = [""] * lenz
    both = [""] * lenz
    
    # First identify the carry and digit gates, because those have known inputs
    for k, v in gates.items():
        # Check for carry gate
        # x[i-1] AND x[i-1] -> carry[i]
        if v[0] == "AND" and checkxy(v):
            carry[int(v[1][1:])+1] = k

        # Check for a digit gate:
        # x[i] XOR y[i] -> digit[i]
        if v[0] == "XOR" and checkxy(v):
            digit[int(v[1][1:])] = k 

    # hack to make some later logic work due to z01 working differently
    both[1] = carry[1]
                    
    # Now identify the chain gates based on digit and carry gates
    for k, v in gates.items():
        # carry[i-1] AND digit[i-1] -> chain[i]
        if v[0] == "AND" and not checkxy(v):
            for i, name in enumerate(digit):
                if name in v[1:]:
                    chain[i+1] = k
                    break
    
    # Now identify the both gates based on chain and carry gates
    for k, v in gates.items():
        # carry[i-1] OR digit[i-1] -> chain[i]
        if v[0] == "OR":
            for i, name in enumerate(carry):
                if name in v[1:]:
                    both[i] = k
                    break

    return {"carry":carry, "digit":digit, "chain":chain, "both":both}

In [249]:
def check_spec( gate, spec ):
    return gate[0] == spec[0] and set(gate[1:]) == set(spec[1:])

def check_digit( gates, design, n ):
    '''check if the 5 gates to identify the nth digit are correct'''
    
    if n == 0:
        if design["digit"][0] != "z00":
            raise BaseException("fixing z00 not implemented")
        return None
    if n == 1:
        if set(gates["z01"][1:]) != set([design["carry"][1], design["digit"][1]]):
            raise BaseException("fixing z01 not implemented")
        return None
    
    # Check chain gate
    chainn = design["chain"][n]
    if len(chainn) == 0:
        raise BaseException("Fix not implemented for missing CHAIN gate name.")
    
    chain_spec = ["AND", design["both"][n-1], design["digit"][n-1]]
    if not check_spec( gates[chainn], chain_spec ):
        return chainn, chain_spec
    
    # Check chain both
    bothn = design["both"][n]
    if len(bothn) == 0:
        raise BaseException("Fix not implemented for missing BOTH gate name.")

    both_spec = ["OR", design["carry"][n], design["chain"][n]]
    if not check_spec( gates[bothn], both_spec ):
        return bothn, both_spec
    
    # Check final calc
    znn = f"z{n:02d}"
    znn_spec = ["XOR", design["digit"][n], design["both"][n]]
    if not check_spec( gates[znn], znn_spec ):
        return znn, znn_spec
    
    return None

def find_first_error( gates, design ):
    '''iterate and find the first error.
    We go least to most significant because later digits rely on having correct names for earlier digits'''
    lenz = len(design["carry"])
    for i in range(0, lenz):
        result = check_digit( gates, design, i )
        if result is not None:
            print("SPEC FOR", i)
            print( "carry", design["carry"][i], gates[ design["carry"][i] ] )
            print( "digit", design["digit"][i], gates[ design["digit"][i] ] )
            print( "chain", design["chain"][i], gates[ design["chain"][i] ] )
            print( "both", design["both"][i], gates[ design["both"][i] ] )
            print( "final", f"z{i:02d}", gates[f"z{i:02d}"] )

            print("BOTH", i-1, ":", design["both"][i-1])
            print("DIGIT", i-1, ":", design["digit"][i-1])
            return result

    return None

In [250]:
def find_gate_by_spec( gates, spec ):
    '''once we id the faulty digit, we want to identify the output that is incorrectly wired to the relevant gate'''
    for k, v in gates.items():
        if v[0] == spec[0] and set(v[1:]) == set(spec[1:]):
            return k

In [274]:
gates = read_gates( puzz_str )
design = identify_gates( gates )

In [275]:
result = find_first_error(gates, design)

SPEC FOR 5
carry kff ('AND', 'x04', 'y04')
digit tvp ('XOR', 'y05', 'x05')
chain rkg ('AND', 'bjc', 'tkj')
both ggh ('OR', 'rkg', 'kff')
final z05 ('OR', 'sgt', 'bhb')
BOTH 4 : tkj
DIGIT 4 : bjc


In [276]:
# z05 isn't even an XOR gate so it's the wrong output for that gate.
find_gate_by_spec( gates, result[1] )

'jst'

In [277]:
# fix this by swapping jst and z05
gates["z05"], gates["jst"] = gates["jst"], gates["z05"]

In [278]:
design = identify_gates( gates )
result = find_first_error(gates, design)

SPEC FOR 10
carry hrq ('AND', 'y09', 'x09')
digit gdf ('XOR', 'y10', 'x10')
chain mcc ('AND', 'mnk', 'cgq')
both tdw ('OR', 'mcc', 'hrq')
final z10 ('XOR', 'mcm', 'tdw')
BOTH 9 : cgq
DIGIT 9 : mnk


In [279]:
# z10 should be digit10 XOR both10. Conclude that digit10 should be mcm instead of gdf
gates["gdf"], gates["mcm"] = gates["mcm"], gates["gdf"]

In [280]:
design = identify_gates( gates )
result = find_first_error(gates, design)

SPEC FOR 15
carry hdb ('AND', 'y14', 'x14')
digit dvj ('XOR', 'x15', 'y15')
chain rkf ('AND', 'kkk', 'jhf')
both vhr ('OR', 'hdb', 'rkf')
final z15 ('AND', 'y15', 'x15')
BOTH 14 : kkk
DIGIT 14 : jhf


In [281]:
# z15 is the wrong type of gate, find the one that's digit15 XOR both15
find_gate_by_spec( gates, result[1] )

'dnt'

In [282]:
gates["dnt"], gates["z15"] = gates["z15"], gates["dnt"]

In [283]:
design = identify_gates( gates )
result = find_first_error(gates, design)

SPEC FOR 30
carry cfp ('AND', 'x29', 'y29')
digit vrg ('XOR', 'x30', 'y30')
chain nqs ('AND', 'fhn', 'fkb')
both kgr ('OR', 'cfp', 'nqs')
final z30 ('AND', 'kgr', 'vrg')
BOTH 29 : fhn
DIGIT 29 : fkb


In [287]:
# Once more, z30 isn't the correct type of gate, so find the logic for z30
swap = find_gate_by_spec( gates, result[1] )
print(swap)
gates["z30"], gates[swap] = gates[swap], gates["z30"]

gwc


In [285]:
design = identify_gates( gates )
result = find_first_error(gates, design)

SPEC FOR 45
carry nng ('AND', 'x44', 'y44')


KeyError: ''

In [ ]:
# This error is because the logic for z45 is different

In [286]:
# INCORRECT!!!
#  Swaps
#z05
#jst
#mcm
#gdf
#z15
#dnt 
#z30
#gwc
#dnt,gdf,gwc,jst,mcm,z05,z15,z30 <-- not sure why this didn't work!
#dnt,gdf,gwc,mcm,jst,z05,z15,z30
#dnt,gdf,gwc,jst,mcm,z05,z15,z30

'gwc'

In [290]:
swaps = ["dnt", "gdf", "gwc", "jst", "mcm", "z05", "z15", "z30"]
swaps.sort()
",".join(swaps)

'dnt,gdf,gwc,jst,mcm,z05,z15,z30'

# Scratch
non-working auto-solver

In [ ]:
def find_first_error( gates, design ):
    '''iterate and find the first error.
    We go least to most significant because later digits rely on having correct names for earlier digits'''
    lenz = len(design["carry"])
    for i in range(0, lenz):
        result = check_digit( gates, design, i )
        print(i, result)
        if result is not None:
            return result

    return None

def fix_design( gates ):
    '''Doesn't work!'''
    while True:
        design = identify_gates( gates )
        result = find_first_error( gates, design )
        # we done!
        if result is None:
            break

        # chain[i] should be correct because it relies on both[i-1] and digit[i-1], which we assume are correct
        if result[0][0] == "z":
            # should be digit[i] XOR both[i]
            if gates[ result[0] ] != "XOR":
                # All Z gates should be XOR, so this is coming from the wrong gate:
                swap1 = find_gate_by_spec( gates, result[1] )
            elif:
                
            
            


        print(result)
        swap0 = result[0]
        swap1 = find_gate_by_spec( gates, result[1] )
        if swap1 is None:
            return design
        
        print(f"Swapping {swap0} and {swap1}")
        gates[swap0], gates[swap1] = gates[swap1], gates[swap0]       
    